In [1]:
import pandas as pd
match_df = pd.read_csv(r"D:\GITHUB\Predictive-Analytics-for-Cricket-Matches-Using-Machine-Learning\source_data\Summary\match_summary\summary_cleaned.csv")
match_df.drop(columns=["Unnamed: 0"], inplace=True)
match_df

,match_id,season,match_date,event_name,gender,match_type,team1,team2,toss_winner,toss_decision,winner,ground
0,1474261,2025,2025-03-01,Bahrain tour of Singapore,male,T20,Singapore,Bahrain,Singapore,bat,Bahrain,"Singapore National Cricket Ground, Singapore"
1,1466424,2025,2025-03-01,ICC Champions Trophy,male,ODI,England,South Africa,England,bat,South Africa,"National Stadium, Karachi"
2,1469311,2025,2025-03-01,Women's Premier League,female,T20,RCB,DC,DC,field,DC,"M Chinnaswamy Stadium, Bengaluru"
3,1474260,2025,2025-02-28,Bahrain tour of Singapore,male,T20,Bahrain,Singapore,Singapore,field,Bahrain,"Singapore National Cricket Ground, Singapore"
4,1469310,2025,2025-02-28,Women's Premier League,female,T20,MI,DC,DC,field,DC,"M Chinnaswamy Stadium, Bengaluru"
...,...,...,...,...,...,...,...,...,...,...,...,...
18453,64815,2003,2003-01-01,India tour of New Zealand,male,ODI,India,New Zealand,India,bat,New Zealand,"Jade Stadium, Christchurch"
18454,64012,2002,2002-12-30,England tour of Australia,male,Test,Australia,England,Australia,bat,Australia,"Melbourne Cricket Ground, Melbourne"
18455,64022,2002,2002-12-29,Pakistan tour of South Africa,male,Test,South Africa,Pakistan,Pakistan,field,South Africa,"Kingsmead, Durban"
18456,64814,2002,2002-12-29,India tour of New Zealand,male,ODI,New Zealand,India,India,field,New Zealand,"McLean Park, Napier"


In [2]:
match_df.columns

Index(['match_id', 'season', 'match_date', 'event_name', 'gender',
       'match_type', 'team1', 'team2', 'toss_winner', 'toss_decision',
       'winner', 'ground'],
      dtype='object')

In [3]:
match_df.isnull().sum()

match_id          0
season            0
match_date        0
event_name       89
gender            0
match_type        0
team1             0
team2             0
toss_winner       0
toss_decision     0
winner            0
ground            0
dtype: int64

In [4]:
match_df["event_name"] = match_df["event_name"].fillna("NA")

In [5]:

from sqlalchemy import create_engine, text
import urllib.parse

# Database credentials
username = 'root'
raw_password = 'Hari@123'
password = urllib.parse.quote(raw_password)  # Encode password
host = 'localhost'
port = '3306'
database = 'cricket_data'

# Create a SQLAlchemy engine
try:
    engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}")
    table_name = 'matches'
    
    # Open a connection to the database
    with engine.connect() as conn:
        # Drop all records from the table if it exists
        truncate_query = f"TRUNCATE TABLE {table_name};"
        conn.execute(text(truncate_query))
        print(f"All records from the '{table_name}' table have been deleted.")
        
        # # Add unique constraint to ADM_NO column (if not already added)
        # alter_query = f"ALTER TABLE {table_name} ADD UNIQUE (ADM_NO);"
        # conn.execute(text(alter_query))
        # print(f"Unique constraint added to ADM_NO in the '{table_name}' table.")

    # Assuming fees_df is your DataFrame containing the data
    # Send DataFrame to MySQL
    match_df.to_sql(name=table_name, con=engine, if_exists='append', index=False)
    print(f"Data successfully inserted into the '{table_name}' table.")

except Exception as e:
    print(f"An error occurred: {e}")


All records from the 'matches' table have been deleted.
Data successfully inserted into the 'matches' table.
